<a href="https://colab.research.google.com/github/radhikasethi2011/ProteinClassify/blob/main/colab/pdb_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

a few checks, 
- some pdb and chains are repeated in the scop list - dropped duplicates. 

In [1]:
!git clone https://github.com/radhikasethi2011/ProteinClassify.git

Cloning into 'ProteinClassify'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [2]:
import numpy as np
import pandas as pd
import os
import glob as glob

In [3]:
cols = ['domid','pdbid', 'pdbchain']
scop = pd.read_csv('/content/ProteinClassify/scop-represented-structures-latest.txt',
                   header = None, skiprows=6, names=cols, delimiter = ' ')
scop

,domid,pdbid,pdbchain
0,8000061,2DT5,B
1,8000376,2FR1,A
2,8000376,6W7S,A
3,8000376,6WH9,A
4,8000376,6WH9,D
...,...,...,...
858211,8107712,7T7E,A
858212,8107712,7T7F,A
858213,8107712,7T7G,A
858214,8107715,7R1K,A


In [4]:
#checking duplicates of pdbid, pdbchain 
#scop[scop.duplicated(['pdbid', 'pdbchain'])]

In [4]:
scop1 = scop.drop_duplicates(subset=['pdbid', 'pdbchain'], keep='first')
scop1.set_index(['pdbid'], inplace=True)
scop1

,domid,pdbchain
pdbid,,
2DT5,8000061,B
2FR1,8000376,A
6W7S,8000376,A
6WH9,8000376,A
6WH9,8000376,D
...,...,...
7QI1,8107705,D
7T7E,8107711,A
7T7F,8107711,A


https://files.rcsb.org/download/2FR1.pdb


In [5]:
%mkdir /content/pdbs
%mkdir /content/ca_csv

In [6]:
pd_list = list(scop1.index)
for i in range(10):
  pd_id = pd_list[i]
  #pd_chain = scop['pdbchain'][i]
  
  print(i, pd_id)
  !wget -q https://files.rcsb.org/download/{pd_id}.pdb -O /content/pdbs/{pd_id}.pdb

0 2DT5
1 2FR1
2 6W7S
3 6WH9
4 6WH9
5 6WH9
6 1A9Y
7 1A9Z
8 1KVQ
9 1KVR


The first element "atom" means this relates to an actual atom in the protein. The 1 relates to a general count, N relates to the type of atom, "MET" is the name of the residue, "A" relates to the the type of chain, 1 (the second "1") is the atom count and then the next 3 numbers are the x-y-z positions in space.



#Understanding

atom | count | type of atom | ressidue name | chain | atom count | x | y  | z
--- | --- | --- | --- | --- | --- | --- | --- | ---
ATOM | 1 | N | MET | A | 1 | 63.645 | 97.355 | 31.526 | 1.00 | 33.80 | N

In [7]:
def is_atom_record(record):
  if record.startswith('ATOM'): 
    return True;
  return False;

def is_intended_chain(record, chain):
  if record[21] == chain: 
    return True
  return False

def is_chain_ter_record(record, chain):
  if record[21] == chain and record.startswith("TER"): 
    return True
  return False

def is_alt_record(record):
  if record[16] == " ": 
    return False
  return True
  
def is_ca_atom(record):
  if record[12:15].strip() == "CA":
    return True
  return False

def parse_atom_records(record):
  atom = record[12:16].strip()
  residue = record[17:20].strip()
  chain = record[21].strip()
  seq_pos = record[22:26].strip()
  x = record[30:38].strip()
  y = record[38:46].strip()
  z = record[46:54].strip()
  return atom, residue, chain, seq_pos, x, y, z

def parse_pdb(contents, chainl): #chain: list
  ca_records = {}
  pos = 0
  for i in range(len(chainl)):
    for line in contents:
      if line.startswith("ENDMDL"): break;
      if is_chain_ter_record(line, chainl[i]): break;

      if not is_atom_record(line): continue
      if not is_intended_chain(line, chainl[i]): continue
      if is_alt_record(line): continue 
      if not is_ca_atom(line): continue

      (atom, residue, chain, rel_pos, x, y, z) = parse_atom_records(line)
      ca_records[pos] = atom, residue, chain, rel_pos, x, y, z
      pos+=1
  return ca_records

def dump_records_to_csv_file(file_name, record_dict):
  file = open(file_name, 'w')
  for key in record_dict.keys():
    (atom, residue, chain, pos, x, y, z) = record_dict[key]
    file.write(f"%s,%s,%s,%s,%s,%s,%s\n"%(atom, residue, chain, pos, x, y, z))


In [8]:
scop1['pdbchain']['2DT5']

'B'

In [9]:
chain = list(scop1['pdbchain']['6WH9'])
print(chain)
print(len(chain))
print(type(chain))

['A', 'D', 'G']
3
<class 'list'>


In [10]:
pdb_list = glob.glob('/content/pdbs/*')
for i in range(len(pdb_list)):
  csv_name = pdb_list[i].split('/')[3].split('.')[0]
  chainl = list(scop1['pdbchain'][csv_name])
  
  print("i: ", i, csv_name, chainl)
  with open(pdb_list[i]) as f: 
    contents = f.readlines() 
  ca_records = parse_pdb(contents, chainl) 
  dump_records_to_csv_file(f'/content/ca_csv/{csv_name}.csv', ca_records)



i:  0 1A9Z ['A']
i:  1 1KVR ['A']
i:  2 6WH9 ['A', 'D', 'G']
i:  3 1A9Y ['A']
i:  4 6W7S ['A']
i:  5 2FR1 ['A']
i:  6 1KVQ ['A']
i:  7 2DT5 ['B']
